In [ ]:
import pandas as pd

import pyteomics.mgf
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import pandas_utils as pu

In [ ]:
outfile_no_filter =  r"C:\git\msn_library\data\acquisition_results\20240527_2batch_no_filter.tsv"
outfile_detected = r"C:\git\msn_library\data\acquisition_results\20240527_2batch_all_lib_only_detected.tsv"
outfile_missing =  r"C:\git\msn_library\data\acquisition_results\20240527_2batch_all_lib_missing.tsv"


In [ ]:
acquisition_pos = {
  "mcebio": r"C:\git\msn_library\library\20240411_mcebio_library_pos_all_lib_MSn.mgf",
  "nihnp": r"C:\git\msn_library\library\20231031_nihnp_library_pos_all_lib_MSn.mgf",
  "mcescaf": r"C:\git\msn_library\library\20231130_mcescaf_library_pos_all_lib_MSn.mgf",
  "otavapep": r"C:\git\msn_library\library\20231130_otavapep_library_pos_all_lib_MSn.mgf",
  "fdadrug": r"C:\git\msn_library\library\20240411_fda_library_pos_all_lib_MSn.mgf",
  "enammol": r"C:\git\msn_library\library\20240527_enamine_molport_pos_all_lib_MSn.mgf",
  "enam": r"C:\git\msn_library\library\20240527_enamine_pos_all_lib_MSn.mgf"
}

acquisition_neg = {
  "mcebio": r"C:\git\msn_library\library\20240411_mcebio_library_neg_all_lib_MSn.mgf",
  "nihnp": r"C:\git\msn_library\library\20231031_nihnp_library_neg_all_lib_MSn.mgf",
  "mcescaf": r"C:\git\msn_library\library\20231130_mcescaf_library_neg_all_lib_MSn.mgf",
  "otavapep": r"C:\git\msn_library\library\20231130_otavapep_library_neg_all_lib_MSn.mgf",
  "fdadrug": r"C:\git\msn_library\library\20240411_fda_library_neg_all_lib_MSn.mgf",
  "enamol": r"C:\git\msn_library\library\20240527_enamine_molport_neg_all_lib_MSn.mgf",
  "enam": r"C:\git\msn_library\library\20240527_enamine_neg_all_lib_MSn.mgf"
}


libraries = {
  "mcebio": r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv",
  "nihnp": r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv",
  "mcescaf": r"C:\git\msn_library\data\iocb_libraries\Radim_mce_complete_cleaned.tsv",
  "otavapep": r"C:\git\msn_library\data\iocb_libraries\iocb_peptide_library_cleaned.tsv",
  "fdadrug": r"C:\git\msn_library\data\iocb_libraries\fda_approved_drugs\FDAlib\FDAlib_cleaned.tsv",
  "enammol": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\enamine_molport_combined.tsv",
  "enam": r"C:\git\msn_library\data\iocb_libraries\radim_enamine\ML9687624_10k_cleaned.tsv"
}

In [ ]:
def read_mgf(infile) -> pd.DataFrame:
  import re
  rows = []
  counter = 0

  with pyteomics.mgf.MGF(infile, encoding='utf-8') as f_in:
    for spectrum_dict in tqdm(f_in):
      if spectrum_dict is not None:
        rows.append(spectrum_dict["params"])
      else:
        counter += 1

  df = pd.DataFrame(rows)
  if "inchikey" not in df.columns:
    df["inchikey"] = df["inchiaux"]
  if "compound_name" not in df.columns:
    df["compound_name"] = df["name"]
  if "usi" in df.columns:
    df["unique_sample_id"] = ["pluskal{}_id".format(re.search(r'pluskal(.*?)_id', usi).group(1)) for
                              usi in df["usi"]]
  return df


def combine_polarity(old, new):
  if old == "both":
    return "both"
  match new:
    case "both":
      return new
    case "positive":
      return "both" if old == "negative" else "positive" 
    case "negative":
      return "both" if old == "positive" else "negative" 
    case _:
      return old

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value).copy()
  df = df[df["monoisotopic_mass"] > 114]
  df["library"] = key
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  dfs.append(df)


metadata_df = pd.concat(dfs, ignore_index=True)


metadata_df

In [ ]:
positive = []
for key, value in acquisition_pos.items():
  df = read_mgf(value)[["inchikey", "unique_sample_id"]].copy()
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  positive.append(df)


positive_df = pd.concat(positive, ignore_index=True)


positive_df

In [ ]:
negative = []
for key, value in acquisition_neg.items():
  df = read_mgf(value)[["inchikey", "unique_sample_id"]].copy()
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  negative.append(df)


negative_df = pd.concat(negative, ignore_index=True)


negative_df

In [ ]:
mapper = {
  "left_only": "positive",
  "right_only": "negative",
  "both": "both",
}

detect = {
  "left_only": "missing",
  "right_only": "error",
  "both": "detected",
}


detected_df = pd.merge(positive_df, negative_df, indicator="polarity", how="outer", on=["unique_sample_id", "inchikey"])
metadata_detected_df = pd.merge(metadata_df, detected_df, how="left", on=["unique_sample_id", "inchikey"], indicator="detected")
metadata_detected_df["polarity"] = [mapper.get(pol, "missing") for pol in metadata_detected_df["polarity"]]
metadata_detected_df["detected"] = [detect.get(ex, "") for ex in metadata_detected_df["detected"]]
metadata_detected_df

In [ ]:
metadata_detected_df[["unique_sample_id", "inchikey", "polarity"]]

In [ ]:
metadata_detected_df.groupby("detected").count()["inchikey"]

In [ ]:
filtered = metadata_detected_df[metadata_detected_df["detected"] == "detected"]
filtered

In [ ]:
missing = metadata_detected_df[metadata_detected_df["detected"] == "missing"]
missing

In [ ]:
pu.save_dataframe(metadata_detected_df, outfile_no_filter)
pu.save_dataframe(filtered, outfile_detected )
pu.save_dataframe(missing, outfile_missing)

## replace lib name

In [ ]:
# df["library"] = df["library"].replace({"01_mce_bioactive":"mcebio", "02_mce_scaffold_library":"mcescaf", "03_nih_natural_product":"nihnp", "04_iocb_peptide":"otavapep"})

In [ ]:
df["library"]

In [ ]:
# pu.save_dataframe(df, file)

In [ ]:
df[df["split_inchikey"] == "WZRCQWQRFZITDX"][["inchikey", "split_inchikey", "compound_name", "synonyms", "polarity", "detected", "unique_sample_id"]]

In [ ]:
detected = pu.read_dataframe(r"C:\git\msn_library\data\acquisition_results\202311_all_lib_only_detected.tsv")
detected

In [ ]:
filtered = detected[["inchikey", "canonical_smiles", "isomeric_smiles", "compound_name", "synonyms", "monoisotopic_mass", "logp", "polarity", "library", "unique_sample_id"]]
filtered

In [ ]:
filtered[filtered.duplicated(["inchikey"], keep=False)].drop_duplicates(["inchikey", "polarity"])

In [ ]:
pu.save_dataframe(filtered, r"C:\git\msn_library\data\acquisition_results\202311_all_lib_only_detected_columns_filtered.tsv")